# Figures S5, S6 & S7

Sensitivity analysis

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sb
from matplotlib import pyplot as plt
from tqdm.auto import tqdm

import analysis

In [2]:
# Register tqdm with pandas for `progress_apply`
tqdm.pandas()

In [3]:
OUTPUT_DIR = Path('figures/fig-s5,s6,s7')

In [4]:
# Significance threshold
ALPHA = 0.05

### Load data

In [5]:
stats = pd.read_parquet("results/stats.parquet").reset_index()

In [6]:
# Rename and order metrics
metric_names = {
    "cityblock": "Manhattan",
    "euclidean": "Euclidean",
    "chebyshev": "Chebyshev",
}
stats["metric"] = stats["metric"].map(metric_names)

assert len(stats["metric"].unique()) == len(metric_names)
assert np.nan not in stats["metric"].unique()

stats["metric"] = pd.Categorical(
    stats["metric"], categories=metric_names.values(), ordered=True
)

In [7]:
# Parameters to iterate over when plotting
GROUPING_PARAMETERS = [
    "window",
    "window_param",
    "overlap",
    "log_frequency",
]

### Plotting functions

In [8]:
x = "state_length"
y = "metric"

In [9]:
width = 5
height = 3
scale = 0.8

In [10]:
def plot_interaction(data):
    params = dict(zip(GROUPING_PARAMETERS, data.name))
    
    fig = plt.figure(figsize=(6.4, 4.8))
    ax = analysis.heatmap(
        data=data,
        x=x,
        y=y,
        value="p.value",
        annot_kws=dict(fontsize=13),
        cmap=sb.color_palette("mako", as_cmap=True),
        cbar=False,
    )
    ax.get_xaxis().set_visible(False)
    ax.set_ylabel("")
    fig.tight_layout()
    analysis.set_ax_size(
        width * scale, height * scale, fig=fig, ax=ax, aspect=True,
    )

    filename = analysis.make_filename(params, prefix=f"sensitivity_analysis", suffix=interaction)
    path = OUTPUT_DIR / filename
    analysis.savefig(
        fig,
        str(path) + ".svg",
        facecolor=None,
        edgecolor=None,
    )
    print(path)

    # Collect combinations that are not below significance threshold
    # so we can omit them in the posthoc tests
    insignificant = data.loc[data["p.value"] >= ALPHA, [x, y]]
    return insignificant


def plot_interaction_sensitivity(interaction):
    insignificant = (
        stats.loc[
            (stats["interaction"] == interaction)
            & (stats["test_type"] == "likelihood_ratio")
            & (stats["term"] == "model")
        ]
        .groupby(by=GROUPING_PARAMETERS, dropna=False)
        .progress_apply(plot_interaction)
    )
    insignificant.index = insignificant.index.droplevel(level=None)
    insignificant["insignificant_likelihood_ratio"] = True
    return insignificant

In [11]:
positions = dict(
    layer=dict(
        suptitle_x=0.2,
        suptitle_y=0.71,
        hspace=-0.83,
    ),
    area=dict(
        suptitle_x=0.2,
        suptitle_y=0.70,
        hspace=-0.89,
    ),
)

In [12]:
def plot_posthoc(data, name=None):
    if name is None:
        name = data.name
    params = dict(zip(GROUPING_PARAMETERS, name))

    g = sb.FacetGrid(
        data=data,
        row=interaction,
        margin_titles=True,
        height=6,
        aspect=1,
        sharex=True,
        sharey=True,
    )

    g.map_dataframe(
        analysis.heatmap,
        x=x,
        y=y,
        value="p.value",
        mask="insignificant_likelihood_ratio",
        cmap=sb.color_palette("mako", as_cmap=True),
        annot_kws=dict(fontsize=13),
        cbar=False,
    )

    for ax in g.axes.flat:
        ax.set_facecolor("#ddd")
    for ax in g.axes.flat[:-1]:
        ax.get_xaxis().set_visible(False)
    g.fig.subplots_adjust(hspace=positions[interaction]["hspace"], wspace=0.0)
    g.fig.tight_layout()
    for ax in g.axes.flat:
        analysis.set_ax_size(
            width * scale, height * scale, fig=g.fig, ax=ax, aspect=True
        )
    g.axes.flat[-1].set_xlabel(x.replace("_", " ") + " (s)")
    g.set_titles(row_template="{row_name}", col_template="{col_name}")

    filename = analysis.make_filename(
        params, prefix=f"sensitivity_analysis", suffix=f"{interaction}__posthoc"
    )
    path = OUTPUT_DIR / filename
    analysis.savefig(
        g.fig,
        str(path) + ".svg",
        facecolor=None,
        edgecolor=None,
    )
    print(path)

    return g


def plot_posthoc_sensitivity(interaction, insignificant):
    (
        stats.loc[
            (stats["interaction"] == interaction) & (stats["test_type"] == "posthoc")
        ]
        .merge(insignificant, on=GROUPING_PARAMETERS + [x, y], how="left")
        .groupby(by=GROUPING_PARAMETERS, dropna=False)
        .progress_apply(plot_posthoc)
    )

# Plot

In [13]:
for interaction in tqdm(["area", "layer"]):
    insignificant = plot_interaction_sensitivity(interaction)
    plot_posthoc_sensitivity(interaction, insignificant)
    plt.close("all")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-False__area
figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-True__area
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-False__area
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-True__area
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-False__area
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-True__area


  0%|          | 0/6 [00:00<?, ?it/s]

figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-False__area__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-True__area__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-False__area__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-True__area__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-False__area__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-True__area__posthoc


  0%|          | 0/6 [00:00<?, ?it/s]

figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-False__layer
figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-True__layer
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-False__layer
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-True__layer
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-False__layer
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-True__layer


  0%|          | 0/6 [00:00<?, ?it/s]

figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-False__layer__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-kaiser__window_param-14.0__overlap-0.5__log_frequency-True__layer__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-False__layer__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-rectangular__window_param-nan__overlap-0.0__log_frequency-True__layer__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-False__layer__posthoc
figures/fig-s5,s6,s7/sensitivity_analysis__window-tukey__window_param-0.25__overlap-0.125__log_frequency-True__layer__posthoc
